# Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO. You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
import pandas as pd
import numpy as np

sales = pd.read_csv('kc_house_data.csv')
sales['intercept'] = np.ones(len(sales))

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['intercept', 'bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Now we can run a Lasso Regression on our dataset.

In [11]:
from sklearn import linear_model

model_all = linear_model.Lasso(alpha=1e8)
model_all.fit(sales[all_features], sales['price'])

Lasso(alpha=100000000.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [12]:
model_all.coef_

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        1.61142560e+02,  0.00000000e+00,  1.18887531e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00])

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

In [15]:
import numpy as np

train_and_validation_mask = np.random.rand(len(sales)) < 0.9
train_and_validation = sales[train_and_validation_mask]
test = sales[~train_and_validation_mask]

train_mask = np.random.rand(len(train_and_validation)) < 0.5
train = train_and_validation[train_mask]
validation = train_and_validation[~train_mask]

(len(train), len(validation), len(test))

(9743, 9656, 2214)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data.
    * Compute the RSS on VALIDATION data
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [29]:
def compute_rss(features, output, model):
    predictions = model.predict(features)
    error = output - predictions
    return np.dot(error, error)

In [30]:
l1_penalties = np.logspace(1, 7, num=13)
models = []
for penalty in l1_penalties:
    model = linear_model.Lasso(alpha=penalty, max_iter=1000)
    model.fit(train[all_features], train['price'])
    models.append(model)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


What was the best value for the `l1_penalty`?

In [34]:
min_error = None
min_l1_penalty = None
min_idx = None

for idx, model in enumerate(models):
    error = compute_rss(validation[all_features], validation['price'], model)
    if min_error is None or error < min_error:
        min_error = error
        min_l1_penalty = l1_penalties[idx]
        min_idx = idx

(min_error, min_l1_penalty, min_idx)

(417756388829315.3, 1000.0, 4)

Also, using this value of L1 penalty, how many nonzero weights do you have?

In [39]:
print('Non-Zero:', np.count_nonzero(models[min_idx].coef_))
print('Total:', len(all_features))

Non-Zero: 15
Total: 18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [40]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [41]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data.
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.

In [47]:
nonzero_counts = []

for l1_penalty in np.logspace(4, 10, num=20):
    model = linear_model.Lasso(alpha=l1_penalty)
    model.fit(train[all_features], train['price'])
    nonzero_counts.append(np.count_nonzero(model.coef_))

nonzero_counts

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[13, 12, 10, 7, 7, 6, 6, 5, 3, 3, 3, 2, 2, 2, 1, 1, 1, 0, 0, 0]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)

In [49]:
l1_penalty_min = None
l1_penalty_max = None

l1_penalties = np.logspace(4, 10, num=20)

for idx, nonzero_count in enumerate(nonzero_counts):
    l1_penalty = l1_penalties[idx]
    if nonzero_count > max_nonzeros:
        l1_penalty_max = l1_penalty
    
    if l1_penalty_min is None and nonzero_count < max_nonzeros:
        l1_penalty_min = l1_penalty

(l1_penalty_min, l1_penalty_max)

(379269.0190732254, 42813.32398719396)

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [50]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data.
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [51]:
best_model = None
best_error = None
best_l1_penalty = None

for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1_penalty)
    model.fit(train[all_features], train['price'])
    nonzero_count = np.count_nonzero(model.coef_)
    if nonzero_count is not max_nonzeros:
        continue # Only want to consider models with a particular sparsity.

    error = compute_rss(validation[all_features], validation['price'], model)
    if best_error is None or error < best_error:
        best_error = error
        best_model = model
        best_l1_penalty = l1_penalty

(best_error, best_l1_penalty)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


(608197683398346.4, 202187.07429110355)

What features in this model have non-zero coefficients?

In [52]:
best_model.coef_ != 0

array([False, False,  True, False,  True, False,  True,  True, False,
       False, False, False, False, False,  True, False,  True,  True])

In [54]:
np.array(all_features)[best_model.coef_ != 0]

array(['bedrooms_square', 'sqft_living', 'sqft_lot', 'sqft_lot_sqrt',
       'sqft_above', 'yr_built', 'yr_renovated'], dtype='<U16')